In [1]:
from ipywidgets import interact
import numpy as np
import pandas as pd
import csv
from IPython.display import Image


import bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models.glyphs import Patch
output_notebook()



Loading BokehJS ...

# Modèle d'atmosphère

On considère un modèle d'atmosphère dans lequel la première couche (troposphère) est une atmosphère adiabatique et la deuxième couche (stratosphère) une atmosphère isotherme.

Au sol, on a $p_0=10^5\;$Pa et une densité de l'air $\rho_0=1.23\;$kg.m$^{-3}$. L'air est un gaz parfait de coefficient $\gamma =1.4$. 

La transition se fait à $h_\text{trans}$

La pression et la masse volumique varient avec l'altitude $h$ selon  $d p = -\rho g\,d h$.

La loi des gaz parfait se réécrit
    
$$T=\frac{pM}{R\rho}$$

La portion adiabatique impose

$$     p\rho^{-\gamma} = \text{cte}=p_0\rho_0^{-\gamma}\quad\Longrightarrow\quad \rho=\rho_0
        \left(\frac{p}{p_0}\right)^{\frac{1}{\gamma}}$$
        
$$       d p = -\rho_0 g \left(\frac{p}{p_0}\right)^{1/\gamma} d h$$

$$      p_\text{tropo}(h) = p_0 \left(1 - \frac{\gamma - 1}{\gamma}\frac{\rho_0g}{p_0}h\right)^\frac{\gamma}{\gamma - 1} = p_0 \left(1-\frac{h}{h_0}\right)^\frac{\gamma}{\gamma - 1}$$
     
$$      h_0 = \frac{\gamma}{\gamma - 1}\frac{p_0}{\rho_0 g}$$    

Et dans la troposphère, on trouve aussi 

$$T_\text{tropo}=T_0\left(1-\frac{h}{h_0}\right)$$

In [3]:
#Constante des gaz parfaite et données au sol
R=8.314
p0=10**5
rho0=1.23
gamma=1.4
T0=300
g=9.81

In [4]:
h0=(gamma/(gamma-1))*p0/(g*rho0)

In [7]:
h0/1000 # h0 en km

29.006406272013795

In [23]:
def ptropo(h):
    ptro=p0*(1-h/h0)**(gamma/(gamma-1))
    return(ptro)

def Ttropo(h):
    ttro = T0*(1-h/h0)
    return(ttro)

In [12]:
h1 = np.linspace(0,29,291)

In [22]:
f=figure(title="évolution de la pression dans le modèle adiabatique",x_axis_label="altitude (km)",
         y_axis_label="pression (bar)", plot_height=400, plot_width=600)
f.line(h1,ptropo(h1*1000)/10**5,color='darkcyan',line_width=4)

show(f)

In [25]:
f=figure(title="évolution de la température dans le modèle adiabatique",x_axis_label="altitude (km)",
         y_axis_label="Température (K)", plot_height=400, plot_width=600)
f.line(h1,Ttropo(h1*1000),color='darkcyan',line_width=4)

show(f)

### Portion isotherme

Dans la portion isotherme, 

$$ d p=-\frac{Mpg}{RT_\text{strat}}d h$$

$$   \ln(p) = -\frac{Mg}{RT_\text{strat}}h + A$$

$$p_\text{strat} = B e^{\left( -\frac{Mg}{RT_\text{strat}}h\right)}$$
      
Le modèle s'applique à partir de $h_\text{trans}$, transition entre la troposphère et la stratosphère (entre 8 et 18 km suivant la lattitude). 

La pression et la températures sont continues, on obtient $p_\text{trans} = p_\text{tropo} (h_\text{trans})$, et $T_\text{strat} = T_\text{tropo} (h_\text{trans})$

$$\frac{M}{R} = \frac{T \rho}{p} =\frac{T_0\rho_0}{p_0}$$

$$p_\text{strat} = B e^{\left( -\frac{T_0\rho_0 g}{p_0 T_\text{strat}}h\right)}$$


In [42]:
def pstrat(h):
    pstr= B * np.e**(-(T0*rho0*g*h)/(p0*Tstrat))
    return(pstr)

### Comparaison du modèle adiabatique et isotherme depuis le sol 

In [51]:
htrans = 0
ptrans = ptropo(htrans)
Tstrat = Ttropo(htrans)
B = ptrans*np.e**(T0*rho0 *g *htrans/(p0*Tstrat))

In [52]:
f=figure(title="évolution de la pression",x_axis_label="altitude (km)",
         y_axis_label="pression (bar)", plot_height=400, plot_width=600)
f.line(h1,ptropo(h1*1000)/10**5,color='darkcyan',line_width=4, legend_label="modèle adiabatique")
f.line(h1,pstrat(h1*1000)/10**5,color='darkorange',line_width=4, legend_label="modèle isotherme")

show(f)

### transition adiabatique -> isotherme à htrans

In [68]:
htrans = 8000
ptrans = ptropo(htrans)
Tstrat = Ttropo(htrans)
B = ptrans*np.e**(T0*rho0 *g *htrans/(p0*Tstrat))

h2 = np.array([h1i for h1i in h1 if h1i <= htrans/1000])
h3 = np.array([h1i for h1i in h1 if h1i >= htrans/1000])

f=figure(title="évolution de la pression",x_axis_label="altitude (km)",
         y_axis_label="pression (bar)", plot_height=400, plot_width=600)
f.line(h1,ptropo(h1*1000)/10**5,color='darkcyan',line_width=1, legend_label="modèle adiabatique")
f.line(h2,ptropo(h2*1000)/10**5,color='darkcyan',line_width=4, legend_label="modèle adiabatique")
f.line(h1,pstrat(h1*1000)/10**5,color='darkorange',line_width=1, legend_label="modèle isotherme")
f.line(h3,pstrat(h3*1000)/10**5,color='darkorange',line_width=4, legend_label="modèle isotherme")

show(f)

In [65]:
h1

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9,
       11. , 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. ,
       12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1,
       13.2, 13.3, 13.4, 13.5, 13.6, 13.7, 13.8, 13.9, 14. , 14.1, 14.2,
       14.3, 14.4, 14.5, 14.6, 14.7, 14.8, 14.9, 15

Notebook by Cécile Hébert (2023).
Except where otherwise noted, the content of this notebook is licensed under MIT licence.